In [152]:
import h5py
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import shap
import lightgbm as lgb 
import optuna
from optuna.samplers import TPESampler
from optuna.integration import LightGBMPruningCallback
from optuna.pruners import MedianPruner
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif, mutual_info_regression
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error

In [133]:
# read data
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        filename = name.split('/')[-1]
        return pandas.DataFrame(f[filename][:], dtype=np.float64)

train = load_data('train')
test  = load_data('test')

print (f'Shape of training data set: {train.shape}')
print (f'Shape of test data set: {test.shape}')

all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]

all = train[all_variables]

electron_truth = train['Truth']
y = train['p_truth_E']
y = y[electron_truth==1]
X = train[all_variables]
X = X[electron_truth==1]
y = y.to_numpy(dtype='float32')

# prepocessing

norm = preprocessing.MinMaxScaler()
sc = preprocessing.StandardScaler()
X = sc.fit_transform(X)
y = np.reshape(y, (-1,1))
y = sc.fit_transform(y)
y = y.ravel()

X = pd.DataFrame(X, columns=all_variables)
# y = pd.DataFrame(y, columns=['p_truth_E'])

print(y)

print (f'Shape of X: {X.shape}')
print (f'Shape of y: {y.shape}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Shape of training data set: (162500, 166)
Shape of test data set: (160651, 164)
[-0.41764802 -0.01699412  0.01510229 ... -0.29044643 -0.21449289
 -1.1436536 ]
Shape of X: (121495, 160)
Shape of y: (121495,)


In [134]:
# feature_selection
def selection_features(X_train, y_train, X_test):
    select = SelectKBest(score_func=mutual_info_regression, k=12)
    select.fit(X_train, y_train)
    
    return select

In [135]:
# select = selection_features(X_train, y_train, X_test)
# X = pd.DataFrame(X)
# names = X.columns.values[select.get_support()]
# scores = select.scores_[select.get_support()]
# names_scores = list(zip(names, scores))
# ns_df = pd.DataFrame(data = names_scores, columns=['Feature_names', 'Feature_scores'])
# #Sort the dataframe for better visualization
# ns_df_sorted = ns_df.sort_values(['Feature_names', 'Feature_scores'], ascending = [False, True])
# print(ns_df_sorted)

In [136]:
# def get_feature_importance():
#     model = SelectKBest(mutual_info_regression, k=12)#选择k个最佳特征
#     X_new = model.fit_transform(X, y)
#     #feature_data是特征数据，label_data是标签数据，该函数可以选择出k个特征 
 
#     print("model shape: ",X_new.shape)
 
#     scores = model.scores_
#     print('model scores:', scores)  # 得分越高，特征越重要
 
#     p_values = model.pvalues_
#     print('model p-values', p_values)  # p-values 越小，置信度越高，特征越重要
 
#     # 按重要性排序，选出最重要的 k 个
#     indices = np.argsort(scores)[::-1]
#     k_best_features = list(X.columns.values[indices[0:12]])
 
#     print('k best features are: ',k_best_features)
    
#     return k_best_features


In [156]:
lgb_variables = ['p_eCluster',
 'p_rawECluster',
 'p_eAccCluster',
 'p_ecore',
 'p_E7x11_Lr2',
 'p_E7x7_Lr2',
 'p_e277',
 'p_eClusterLr2',
 'p_E5x7_Lr2',
 'p_e255',
 'p_e237',
 'p_e235']



X_lgb = train[lgb_variables]
X_lgb = X_lgb[electron_truth==1]
X_lgb = sc.fit_transform(X_lgb)
X_lgb = pd.DataFrame(X_lgb, columns=lgb_variables)


 

In [165]:
from lightgbm import LGBMRegressor
# predict

# optimize hyperparameters of lgb

def objective_lgb(trial):

    X_lgb_train, X_lgb_test, y_lgb_train, y_lgb_test = train_test_split(X_lgb, y, test_size=0.2, random_state=42)
    train_data = lgb.Dataset(X_lgb_train, label=y_lgb_train)
    valid_data = lgb.Dataset(X_lgb_test, label=y_lgb_test)
    
    boosting_types = ["gbdt", "rf", "dart"]
    boosting_type = trial.suggest_categorical("boosting_type", boosting_types)

    params = {
        'objective': 'regression',
        'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "rf", "dart"]),
        'max_depth': trial.suggest_int('max_depth', 2, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 0, 1000),
        'metric': 'mae',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 0.95, step=0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000, step=20),
        'bagging_freq': trial.suggest_categorical('bagging_freq', [1]),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.95, step=0.1),
        'reg_alpha': trial.suggest_float("reg_alpha", 0, 100, step=0.1),
        'reg_lambda': trial.suggest_int("reg_lambda", 0, 1000, step=1),
        'verbosity': -1,
    }

    lgbm = LGBMRegressor(**params)
    lgbm.fit(X_lgb_train, y_lgb_train, eval_set=[(X_lgb_test, y_lgb_test)],early_stopping_rounds=100, verbose=False)
    pred_lgb=lgbm.predict(X_lgb_test)
    mae = mean_absolute_error(y_lgb_test, pred_lgb)
    return mae


study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_warmup_steps=50),
) 

study.optimize(objective_lgb, n_trials=100, show_progress_bar=True)

study.best_trial.params

[I 2022-05-19 23:43:03,835] A new study created in memory with name: no-name-e05c1148-8079-4c57-8163-4299e95a0c5f
Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  0%|          | 0/100 [00:00<?, ?it/s]The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


  1%|          | 1/100 [00:00<00:23,  4.22it/s]The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[I 2022-05-19 23:43:04,071] Trial 0 finished with value: 0.18356512962289062 and parameters: {'boosting_type': 'rf', 'max_depth': 8, 'min_child_samples': 156, 'learning_rate': 0.08643731496473929, 'feature_fraction': 0.2, 'num_leaves': 2600, 'bagging_freq': 1, 'bagging_fraction': 0.6000000000000001, 'reg_alpha': 70.8, 'reg_lambda': 20}. Best is trial 0 with value: 0.18356512962289062.
[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


  2%|▏         | 2/100 [00:00<00:23,  4.26it/s]The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
Early stopping is not available in dart mode


...

 99%|█████████▉| 99/100 [00:25<00:00,  3.61it/s]The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[I 2022-05-19 23:43:29,292] Trial 98 finished with value: 0.15738991406650638 and parameters: {'boosting_type': 'dart', 'max_depth': 2, 'min_child_samples': 53, 'learning_rate': 0.4305802593374367, 'feature_fraction': 0.8, 'num_leaves': 620, 'bagging_freq': 1, 'bagging_fraction': 0.2, 'reg_alpha': 48.5, 'reg_lambda': 999}. Best is trial 93 with value: 0.30486816466592226.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


100%|██████████| 100/100 [00:25<00:00,  3.90it/s]

[I 2022-05-19 23:43:29,502] Trial 99 finished with value: 0.28912059581752136 and parameters: {'boosting_type': 'rf', 'max_depth': 2, 'min_child_samples': 129, 'learning_rate': 0.48466631065504984, 'feature_fraction': 0.9, 'num_leaves': 820, 'bagging_freq': 1, 'bagging_fraction': 0.30000000000000004, 'reg_alpha': 59.5, 'reg_lambda': 958}. Best is trial 93 with value: 0.30486816466592226.


{'boosting_type': 'rf',
 'max_depth': 2,
 'min_child_samples': 74,
 'learning_rate': 0.4319464280296327,
 'feature_fraction': 0.9,
 'num_leaves': 680,
 'bagging_freq': 1,
 'bagging_fraction': 0.2,
 'reg_alpha': 63.5,
 'reg_lambda': 985}

In [173]:
def optimized_lgb():
    X_lgb_train, X_lgb_test, y_lgb_train, y_lgb_test = train_test_split(X_lgb, y, test_size=0.2, random_state=42)
    train_data = lgb.Dataset(X_lgb_train, label=y_lgb_train)
    valid_data = lgb.Dataset(X_lgb_test, label=y_lgb_test)

    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'rf',
        'max_depth': 2,
        'min_child_samples': 74,
        'learning_rate': 0.4319464280296327,
        'feature_fraction': 0.9,
        'num_leaves': 680,
        'bagging_freq': 1,
        'bagging_fraction': 0.2,
        'reg_alpha': 63.5,
        'reg_lambda': 985,
        'verbosity': -1,
    }

    lgbm = LGBMRegressor(**params)
    lgbm.fit(X_lgb_train, y_lgb_train, eval_set=[(X_lgb_test, y_lgb_test)],early_stopping_rounds=100, verbose=False)
    pred_lgb=lgbm.predict(X_lgb_test)
    mae = mean_absolute_error(y_lgb_test, pred_lgb)
    # y_pred = np.around(lgbm.predict(X_lgb_test.values))
    # acc = accuracy_score(y_pred, y)
    return mae

optimized_lgb()

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


0.30486816466592226